<img src="img/mmdet-logo.png">

# MMDetection 2.0 Tutorials

<a href="" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MMClassification 2.0 Repo**：[https://github.com/open-mmlab/mmdetection/tree/3.x](https://github.com/open-mmlab/mmdetection/tree/3.x)

**MMClassification 官方文档链接**：[https://mmdetection.readthedocs.io/en/3.x/index.html](https://mmdetection.readthedocs.io/en/3.x/index.html)

**MMClassification 2.0 教学视频**：Coming Soon~

**MMClassification 2.0 代码库介绍视频**：Coming Sonn~

**加入微信社群方式**：关注公众号，选择 “加入我们” -> “微信社区”，即可获取入群二维码。非常期待你的到来呀~

**作者**：OpenMMLab

---

## Outline
0. Installation
1. Inference
2. Training
3. Testing
4. Training on Custom Datasets
5. Useful Tools
6. Model Deployment

---

## 0. Installation

**Step 1. 加载预置环境**

我们已经在集群上预装了 OpenMMLab 2.0 环境，使用见 [1-SetupEnvironment.md#1-使用预置环境](1-SetupEnvironment.md#1-使用预置环境)

**Step 2. 下载 MMDetection 源码**

虽然环境里面预装了一系列工具包，但我们也可以使用github上最新的代码。例如可以通过以下方式“覆盖”预装的代码：

In [ ]:
# 使用国内的镜像
!git clone https://gitee.com/open-mmlab/mmdetection.git -b 3.x
# 重要，保证使用的是 clone 下来的版本
%cd mmdetection
# 重要，确保是 v2.0 版本
!git checkout 3.x

**Step 3. 查看安装是否正确**

In [ ]:
import mmdet
print(mmdet.__version__)
print(mmdet.__file__)

---

## 1. Inference 
用 MMDetection [模型库](https://mmdetection.readthedocs.io/en/3.x/model_zoo.html)中预训练好的模型对图片进行预测。

**Step 1. 准备一张图片 `mmdet-demo.jpg`**

<img src="img/mmdet-demo.jpg">

**Step 2. 下载模型以及对应的配置文件**

```bash
mim download ${PACKAGE} ${CONFIG_FILENAME} --dest ${DESTINATION_PATH}
```

训练工具 `mim download` 参数讲解：

+ PACKAGE：工具包名
+ CONFIG_FILENAME：配置文件的文件名。**注意：不要加 `.py`**
+ DESTINATION_PATH：下载文件保存路径

更多请参考：[链接](https://openmim.readthedocs.io/en/latest/api.html#module-mim.commands.download)

In [ ]:
!mim download mmdet --config rtmdet_s_8xb32-300e_coco --dest tutorial/

**Step 3. 用工具进行预测，解析推理结果 `result`，并可视化预测结果**

```bash
python demo/image_demo.py \
    ${IMG_PATH} \
    ${CONFIG_FILE} \
    ${CHECKPOINT_PATH} \
    [--device ${YOUR_DEVICE}] \
    [--out-file ${OUTPUT_PATH}] \
    [--palette ${coco,voc,citys,random,none}] \
    [--score-thr ${THRESHOLD_SCORE}]
```

推理工具 [demo/image_demo.py](https://github.com/open-mmlab/mmclassification/blob/1.x/demo/image_demo.py) 参数讲解:
+ IMG_PATH：测试图片所在路径

+ CONFIG_FILE：配置文件所在路径

+ CHECKPOINT_PATH：模型文件所在路径

+ device：设备，默认 'cuda:0'

+ out-file : 输出文件路径及文件名

+ palette : 可视化颜色方案选择

+ score-thr : 检测框阈值设定

In [ ]:
# 在管理节点用 CPU执行
!python demo/image_demo.py demo/demo.jpg tutorial/rtmdet_s_8xb32-300e_coco.py --weights tutorial/rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth --out-dir tutorial/demo/ --device cpu
# 在GPU节点执行
!srun -p gpu -n 1 --gres gpu:1 python demo/image_demo.py demo/demo.jpg tutorial/rtmdet_s_8xb32-300e_coco.py --weights tutorial/rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth --out-dir tutorial/demo/

---

## 2. Training
用 MMDetection 在 Pascal VOC 2007 & 2012 数据集上训练检测模型 RetinaNet

**Step 1. 准备数据集**

[Pascal VOC 2007](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/) 数据集包含 20 个类别共 9963 张图片，其中 5011 张作为训练图片，4952 张作为测试图片。

<img src="img/Pascal_VOC_2007_dataset.png">

我们推荐在 MMDetection 代码库的根目录下新建名为 `data` 的文件夹，并把数据集放在 `data` 文件夹里。

MMDetection 提供了数据集[下载脚本](https://github.com/open-mmlab/mmdetection/blob/3.x/tools/misc/download_dataset.py)

```bash
python tools/misc/download_dataset.py \
    [--dataset-name ${DATASET_NAME}] \
    [--save-dir ${SAVING_DIRECTORY}] \
    [--unzip ${WHETHER_UNZIP}] \
    [--delete ${DELETE_DOWNLOAD_FILE}]
```

参数讲解:

+ dataset-name：数据集名称，默认为 coco2017

+ save-dir : 存储路径，默认为 data/coco

+ unzip : 是否解压缩下载文件

+ delete : 是否删除下载的压缩文件

更多准备数据集的详情请参考：[链接](https://mmdetection.readthedocs.io/en/3.x/user_guides/dataset_prepare.html)

In [ ]:
# !python tools/misc/download_dataset.py --dataset-name voc2007 --save-dir data --unzip
# 集群上已经下载，位于 /cluster/share/VOCdevkit/

由于下载工具还不支持 Pascal VOC 2012，需访问 [Pascal VOC 2012 官网](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/)自行下载。

In [ ]:
# 为了简化数据准备流程，建议使用集群上已经下载的版本，通过符号链接链接到 data 文件夹下
!mkdir -p ~/mmdetection/data
!ln -s /cluster/share/VOCdevkit/ ~/mmdetection/data/VOCdevkit

**Step 2. 准备 RetinaNet 模型的配置文件**

模型配置文件 `retinanet_r50_fpn_1x_voc0712.py` 包含在源代码中，位于 `configs/pascal_voc/` 下

**Step 3. 解读配置文件** 

+ param_scheduler：参数调度器
+ optim_wrapper：优化器
+ train_cfg、val_cfg、test_cfg：训练、验证和测试的循环设置，比如训练多少个 epoch `max_epochs` 等等
+ auto_scale_lr : 自动设置学习率


**Step 4. 用工具开始训练**

```bash
mim train ${PACKAGE} ${CONFIG_FILE} --gpus ${GPUS}
```

训练工具 `mim train` 参数讲解：

+ PACKAGE：工具包名
+ CONFIG_FILE：配置文件所在路径
+ GPUS：使用 GPU 的数量，默认值为 1

更多请参考：[链接](https://openmim.readthedocs.io/en/latest/api.html#module-mim.commands.train)


我们复现训练的结果：
+ Epoch 1's mAP：0.063
+ Epoch 2's mAP：0.353
+ Epoch 3's mAP：0.552
+ Epoch 4's mAP：0.649

In [ ]:
# 仅支持下载源码后使用
# 单卡训练
!srun -p gpu -N 1 -n 1 -c 4 --gres gpu:1 python -u tools/train.py configs/pascal_voc/retinanet_r50_fpn_1x_voc0712.py --auto-scale-lr
# 多卡训练
!srun -p gpu -N 1 -n 2 -c 4 --gres gpu:2 python -u tools/train.py configs/pascal_voc/retinanet_r50_fpn_1x_voc0712.py --auto-scale-lr --launcher slurm

# 下载或没下载源码都可使用、多卡训练
!mim train mmdet -l slurm -g 2 -G 2 -c 4 -p gpu configs/pascal_voc/retinanet_r50_fpn_1x_voc0712.py --auto-scale-lr

**Step 5. 理解打印出来的 log 日志文件**
+ 系统环境
+ 配置文件
+ 训练过程中学习率的变化、loss 变化、消耗时间、消耗内存
+ 验证集上的准确率

---

## 3. Testing
用 MMDetection 在 Pascal VOC 2007 & 2012 数据集上测试分类模型 RetinaNet

**Step 1. 准备数据集**

我们依旧使用 [Pascal VOC 2007 & 2012](http://host.robots.ox.ac.uk/pascal/VOC/) 数据集来演示

**Step 2. 下载 RetinaNet 模型以及对应的配置文件**

+ 使用上一节准备好的 RetinaNet 配置文件

+ 并下载对应的 RetinaNet  模型

In [ ]:
!wget -P tutorial https://download.openmmlab.com/mmdetection/v2.0/pascal_voc/retinanet_r50_fpn_1x_voc0712/retinanet_r50_fpn_1x_voc0712_20200617-47cbdd0e.pth

**Step 3. 用工具开始测试**

```bash
mim test ${PACKAGE} ${CONFIG_FILE} --checkpoint ${CHECKPOINT_FILE} --out ${DUMP_FILE}
```

训练工具 `mim test` 参数讲解：

+ PACKAGE：工具包名
+ CONFIG_FILE：配置文件所在路径
+ CHECKPOINT_FILE：模型文件所在路径
+ OTHER_ARGS: 其他和各自工具包（代码库）相关的参数。比如 MMDetection 中的
    + DUMP_FILE：保存模型对每个样本的预测结果文件所在路径。默认不设置该参数，也就是不会保存该文件。如果要保存该文件，文件命名必须以 .pkl 为后缀


更多请参考：[链接](https://openmim.readthedocs.io/en/latest/api.html#module-mim.commands.test)

In [ ]:
# 下载了源码，2卡测试
!srun -p gpu -N 1 -n 2 -c 4 --gres gpu:2 python -u tools/test.py configs/pascal_voc/retinanet_r50_fpn_1x_voc0712.py tutorial/retinanet_r50_fpn_1x_voc0712_20200617-47cbdd0e.pth --out tutorial/result_voc.pkl --launcher slurm
# 没下载源码，2卡测试
!mim test mmdet -l slurm -g 2 -G 2 -c 4 -p gpu configs/pascal_voc/retinanet_r50_fpn_1x_voc0712.py --checkpoint tutorial/retinanet_r50_fpn_1x_voc0712_20200617-47cbdd0e.pth --out tutorial/result_voc.pkl

**解读评估设置与结果**

```python
val_evaluator = dict(type='VOCMetric', metric='mAP', eval_mode='11points')
test_evaluator = val_evaluator
```
Pascal VOC 2007 使用 `11points` 作为默认的评估模式，而Pascal VOC 2012 默认使用 `area`

除了 `recall`、`AP`、`mAP`等，MMDetection 还支持许多其他评估指标（详情请参考：[链接](hhttps://mmdetection.readthedocs.io/en/3.x/api.html#mmdet-evaluation)），我们只需要修改配置文件中的 `val_evaluator` 和 `test_evaluator` 里的内容即可。

---

## 4. Train on Custom Datasets

使用自定义数据集训练一个新的检测模型通常有三个步骤：
1. 准备一个自定义数据集
2. 准备一个配置文件
3. 在自定义数据集上完成训练


**Step 1. 准备数据集**

演示使用的自定义数据集为[balloon dataset](https://github.com/matterport/Mask_RCNN/tree/master/samples/balloon)

MMDetection支持三种自定义数据集的准备方式：

+ 将自定义数据集转换为COCO格式
+ 将自定义数据集转换为中间格式
+ 实现一个新的数据集

前两种方式通常会比第三种方式简单，因此建议使用前两种方式

在这个例子中，我们将转换为COCO格式


下载balloon数据集放入data文件夹

In [ ]:
# 集群上已经下载该数据集，位于 /cluster/share/balloon/ 下，不需要重复下载解压
# !wget -P data https://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip

解压缩balloon数据集

In [ ]:
# 集群上已经下载该数据集，位于 /cluster/share/balloon/ 下，可以不需要重复下载解压
!unzip /cluster/share/balloon/balloon_dataset.zip -d data/

In [ ]:
# 为了减轻网络负担，建议使用集群上已经下载的版本，解压到 data 文件夹下
# !mkdir -p ~/mmdetection/data
# !ln -s /cluster/share/balloon/ ~/mmdetection/data/balloon

查看balloon数据集的目录

In [ ]:
# !apt-get install tree # 如果环境中没有 tree 才需要安装
!tree -d data/ballon

查看balloon数据集中的图片

In [ ]:
import mmcv
import matplotlib.pyplot as plt

img = mmcv.imread('data/balloon/train/120853323_d4788431b9_b.jpg')
plt.figure(figsize=(15, 10))
plt.imshow(mmcv.bgr2rgb(img))
plt.show()

查看balloon数据集的标注格式

In [ ]:
!cat data/balloon/train/via_region_data.json | python -m json.tool

对比COCO标注格式

```json
{
    "images": [image],
    "annotations": [annotation],
    "categories": [category]
}

image = {
    "id": int,
    "width": int,
    "height": int,
    "file_name": str,
}

annotation = {
    "id": int,
    "image_id": int,
    "category_id": int,
    "segmentation": RLE or [polygon],
    "area": float,
    "bbox": [x,y,width,height], # (x, y) are the coordinates of the upper left corner of the bbox
    "iscrowd": 0 or 1,
}

categories = [{
    "id": int,
    "name": str,
    "supercategory": str,
}]
```

新建一个 balloon 数据转换文件 `balloon_converter.py` 粘贴代码并运行，将 balloon 数据集原有标注转换为 COCO 的标注格式

In [ ]:
!touch tutorial/balloon_converter.py

In [ ]:
import os.path as osp

import mmcv

from mmengine.fileio import dump, load
from mmengine.utils import track_iter_progress


def convert_balloon_to_coco(ann_file, out_file, image_prefix):
    data_infos = load(ann_file)

    annotations = []
    images = []
    obj_count = 0
    for idx, v in enumerate(track_iter_progress(data_infos.values())):
        filename = v['filename']
        img_path = osp.join(image_prefix, filename)
        height, width = mmcv.imread(img_path).shape[:2]

        images.append(
            dict(id=idx, file_name=filename, height=height, width=width))

        for _, obj in v['regions'].items():
            assert not obj['region_attributes']
            obj = obj['shape_attributes']
            px = obj['all_points_x']
            py = obj['all_points_y']
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            x_min, y_min, x_max, y_max = (min(px), min(py), max(px), max(py))

            data_anno = dict(
                image_id=idx,
                id=obj_count,
                category_id=0,
                bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                area=(x_max - x_min) * (y_max - y_min),
                segmentation=[poly],
                iscrowd=0)
            annotations.append(data_anno)
            obj_count += 1

    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{
            'id': 0,
            'name': 'balloon'
        }])
    dump(coco_format_json, out_file)


if __name__ == '__main__':
    convert_balloon_to_coco(ann_file='data/balloon/train/via_region_data.json',
                            out_file='data/balloon/train/annotation_coco.json',
                            image_prefix='data/balloon/train')
    convert_balloon_to_coco(ann_file='data/balloon/val/via_region_data.json',
                            out_file='data/balloon/val/annotation_coco.json',
                            image_prefix='data/balloon/val')


In [ ]:
!python tutorial/balloon_converter.py

**Step 2. 准备配置文件**

下载 `mask-rcnn_r50_fpn_1x_coco.py` 配置文件，并拷贝重命名为 `balloon`

也可以尝试其他配置文件，如 `mask-rcnn_r50_fpn_mstrain-poly_3x_coco.py` 等，以比较不同的效果

In [ ]:
!mim download mmdet --config mask-rcnn_r50_fpn_1x_coco --dest tutorial/
!cp tutorial/mask-rcnn_r50_fpn_1x_coco.py tutorial/balloon.py

更改配置
- 将检测头中的类别数改为 `1`
    ```python
    model = dict(
        roi_head=dict(
            bbox_head=dict(num_classes=1), mask_head=dict(num_classes=1)))
    ```
- 更改数据集相关配置
    ```python
    data_root = 'data/balloon/'
    metainfo = {
        'classes': ('balloon', ),
        'palette': [
            (220, 20, 60),
        ]
    }
    train_dataloader = dict(
        batch_size=1,
        dataset=dict(
            data_root=data_root,
            metainfo=metainfo,
            ann_file='train/annotation_coco.json',
            data_prefix=dict(img='train/'))) # 图像应位于data_root/data_prefix下面
    val_dataloader = dict(
        dataset=dict(
            data_root=data_root,
            metainfo=metainfo,
            ann_file='val/annotation_coco.json',
            data_prefix=dict(img='val/')))
    test_dataloader = val_dataloader
    ```
- 更改评价指标相关配置
    ```python
    val_evaluator = dict(ann_file=data_root + 'val/annotation_coco.json')
    test_evaluator = val_evaluator
    ```
- 使用预训练的 Mask RCNN 模型
    ```python
    load_from = 'tutorial/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth'
    ```
- 更改 log 打印间隔为 `10`
    ```python
    default_hooks = dict(logger=dict(type='LoggerHook', interval=10))
    ```
- 更改学习率为 `0.02 / 8`
    ```python
    optim_wrapper = dict(optimizer=dict(lr=0.02 / 8))
    ```
- 更改验证间隔为 `4`
    ```python
    train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=12, val_interval=4)
    ```
- 更改权重存储间隔为 `4`
    ```python
    default_hooks = dict(checkpoint=dict(type='CheckpointHook', interval=4))

**Step 3. 使用工具开始训练**

In [ ]:
!mim train mmdet -l slurm -g 2 -G 2 -c 4 -p gpu tutorial/balloon.py

**Step 4. 理解打印出来的 log 日志文件**
+ 系统环境
+ 配置文件
+ 检查模型初始化记录
+ 训练过程中学习率的变化、loss 变化、消耗时间、消耗内存
+ 验证集上的结果

**Step 5. 测试训练的模型，并保存结果文件**

In [ ]:
!mim test mmdet -l slurm -g 2 -G 2 -c 4 -p gpu tutorial/balloon.py --checkpoint work_dirs/balloon/epoch_12.pth --out tutorial/result_balloon.pkl

**Step 6. 测试训练好的模型**

In [ ]:
!python demo/image_demo.py data/balloon/train/7178882742_f090f3ce56_k.jpg tutorial/balloon.py --weights work_dirs/balloon/epoch_12.pth --out-dir tutorial/balloon_test/ --device cpu

---

## 5. Useful Tools

**运行工具 `mim run`**

```bash
mim run ${PACKAGE} ${COMMAND} --yes ${YES} --other_args ${OTHER_ARGS}
```
参数讲解：
+ PACKAGE：工具包名
+ COMMAND：运行命令的名字
+ YES：不要求确认。bool类型的变量，默认为 True
+ OTHER_ARGS: 其他传入各工具包运行命令的参数

更多请参考：[链接](https://openmim.readthedocs.io/en/latest/api.html#module-mim.commands.run)

如果想查看某个工具包支持哪些命令以及使用方法，可使用命令：

```bash
mim run ${PACKAGE} -h
```

比如，查看 MMDetection 目前支持的命令：

In [ ]:
!mim run mmdet -h

### 5.1 Log Analysis
根据训练log文件绘制`loss/mAP`曲线

**Step 1. 安装依赖 `seaborn`**

In [ ]:
# !pip install seaborn

**Step 2. 绘制图像**
```bash
mim run mmdet analyze_logs plot_curve [--keys ${KEYS}] [--title ${TITLE}] [--legend ${LEGEND}] [--style ${STYLE}] [--out ${OUT_FILE}]
```
参数讲解：
+ KEYS：需要绘制的所有参数，默认为`bbox_mAP`
+ TITLE：图像标题
+ LEGEND：图例
+ STYLE：图像风格，默认为`dark`
+ OUT_FILE：保存文件

详情请参考：[链接](https://mmdetection.readthedocs.io/en/3.x/user_guides/useful_tools.html#log-analysis)

In [ ]:
!mim run mmdet analyze_logs plot_curve ${LOG_FILE} --keys loss_cls loss_bbox loss_mask lr time --legend loss_cls loss_bbox loss_mask lr time --out tutorial/log_analysis.png

### 5.2 Result Analysis (有 Bug)
计算每张图像的mAP并根据预测结果保存或展示分数最高和最低的图像

```bash
mim run mmdet analyze_results ${CONFIG} ${PREDICTION_PATH} ${SHOW_DIR} [--show] [--wait-time ${WAIT_TIME}] [--topk ${TOPK}] [--show-score-thr ${SHOW_SCORE_THR}]
```
参数讲解：
+ CONFIG：模型配置文件路径
+ PREDICTION_PATH：训练/测试输出的pickle格式结果文件
+ SHOW_DIR：绘制的GT和检测图像的保存路径
+ --show：是否展示绘制的图像，默认为`False`
+ WAIT_TIME：展示间隔，`0`为阻隔，默认为`0`
+ TOPK：排序后保存的分数最高和最低的图像个数，默认为`20`
+ SHOW_SCORE_THR：展示的分数阈值，默认为`0`

详情请参考：[链接](https://mmdetection.readthedocs.io/en/3.x/user_guides/useful_tools.html#result-analysis)

由于 `evaluation` 程序调整，需要修改一部分 `analyze_results` 代码
    
```python
    if task == 'det':
        # gt_instances = InstanceData()               
        # gt_instances.bboxes = results[index]['gt_instances']['bboxes']
        # gt_instances.labels = results[index]['gt_instances']['labels']

        pred_instances = InstanceData()
        pred_instances.bboxes = results[index]['pred_instances']['bboxes']
        pred_instances.labels = results[index]['pred_instances']['labels']
        pred_instances.scores = results[index]['pred_instances']['scores']

        data_samples = DetDataSample()
        data_samples.pred_instances = pred_instances
        # data_samples.gt_instances = gt_instances
```

In [ ]:
# !mim run mmdet analyze_results tutorial/balloon.py tutorial/result_balloon.pkl tutorial/balloon_results
!python tools/analysis_tools/analyze_results.py tutorial/balloon.py tutorial/result_balloon.pkl tutorial/balloon_results

### 5.3 Browse Dataset
直观地预览数据集（包括图像和边界框），并可以将图像保存到指定目录

```bash
mim run mmdet browse_dataset ${CONFIG} [--output-dir ${OUTPUT_DIR}] [--not-show] [--show-interval ${SHOW_INTERVAL}]
```
参数讲解：
+ CONFIG：模型配置文件路径
+ OUTPUT_DIR：需要保存时的存储路径
+ --not-show：是否展示，默认为`False`
+ SHOW_INTERVAL：展示间隔，默认为`2`

详情请参考：[链接](https://mmdetection.readthedocs.io/en/3.x/user_guides/useful_tools.html#visualization)

In [ ]:
!mim run mmdet browse_dataset tutorial/balloon.py --output-dir tutorial/balloon_vis --not-show

### 5.4 Error Analysis （有 Bug)
按类别或其他标准分析COCO结果，并可将结果绘制成图像

**Step 1. 修改配置文件**
为了得到`bbox`和`segmentation`结果的`json`文件，需要对配置文件中的`test_evaluator`进行修改
修改后的配置为
```python
test_evaluator = dict(
    type='CocoMetric',
    format_only=True, 
    metric=['bbox', 'segm'],
    ann_file=data_root + 'val/annotation_coco.json',
    outfile_prefix='tutorial/balloon_error/balloon'
)
```
其中`format_only`用于指定`json`输出，`ann_file`用于给定标注文件，`outfile_prefix`用于制定输出文件的前缀

**Step 2. 使用测试工具得到json结果**

In [ ]:
!mim test mmdet tutorial/balloon.py --checkpoint work_dirs/balloon/epoch_12.pth

**Step 3. 修改`coco_error_analysis`工具**
因为我们使用了自定义的数据集进行模型的训练和测试，所以需要对`coco_error_analysis`工具进行一些调整
```python
    # child_catIds = gt.getCatIds(supNms=[nm['supercategory']])
    # for idx, ann in enumerate(gt.dataset['annotations']):
    #     if ann['category_id'] in child_catIds and ann['category_id'] != catId:
    #         gt.dataset['annotations'][idx]['ignore'] = 1
    #         gt.dataset['annotations'][idx]['iscrowd'] = 1
    #         gt.dataset['annotations'][idx]['category_id'] = catId
```
注去如上部分的代码

**Step 4. 使用`coco_error_analysis`工具进行错误分析**

```bash
mim run mmdet coco_error_analysis ${RESULT} ${OUT_DIR} [-h] [--ann ${ANN}] [--types ${TYPES[TYPES...]}]
```
参数讲解：
+ RESULT：测试结果文件
+ OUT_DIR：保存结果分析图像的路径
+ ANN：标注文件
+ TYPES：结果格式，默认为`bbox`

详情请参考：[链接](https://mmdetection.readthedocs.io/en/3.x/user_guides/useful_tools.html#error-analysis)

得到`bbox`的分析结果

In [ ]:
# !mim run mmdet coco_error_analysis tutorial/balloon_error/balloon.bbox.json tutorial/balloon_error --ann data/balloon/val/annotation_coco.json
!python tools/analysis_tools/coco_error_analysis.py tutorial/balloon_error/balloon.bbox.json tutorial/balloon_error --ann data/balloon/val/annotation_coco.json

得到`segmentation`的分析结果

In [ ]:
# !mim run mmdet coco_error_analysis tutorial/balloon_error/balloon.segm.json tutorial/balloon_error --ann data/balloon/val/annotation_coco.json --types segm
!python tools/analysis_tools/coco_error_analysis.py tutorial/balloon_error/balloon.segm.json tutorial/balloon_error --ann data/balloon/val/annotation_coco.json --types segm

### 5.5 Model Complexity
计算给定模型的FLOPs和参数量

```bash
mim run mmdet get_flops ${CONFIG_FILE} [--shape ${INPUT_SHAPE}]
```
参数讲解：
+ CONFIG_FILE：模型配置文件
+ INPUT_SHAPE：指定输入形状

详情请参考：[链接](https://mmdetection.readthedocs.io/en/3.x/user_guides/useful_tools.html#model-complexity)

In [ ]:
!mim run mmdet get_flops tutorial/balloon.py

### 5.6 Confusion Matrix
析预测结果并绘制混淆矩阵

```bash
mim run mmdet confusion_matrix ${CONFIG}  ${DETECTION_RESULTS}  ${SAVE_DIR} --show
```
参数讲解：
+ CONFIG：模型配置文件
+ IDETECTION_RESULTS：测试结果文件
+ SAVE_DIR：储存路径
+ --show：是否展示

详情请参考：[链接](https://mmdetection.readthedocs.io/en/3.x/user_guides/useful_tools.html#confusion-matrix)

先绘制balloon数据集的结果

In [ ]:
!mim run mmdet confusion_matrix tutorial/balloon.py tutorial/result_balloon.pkl tutorial/balloon_confusion

由于balloon数据集不包含多类别，再进行voc数据集结果的分析

In [ ]:
!mim run mmdet confusion_matrix configs/pascal_voc/retinanet_r50_fpn_1x_voc0712.py tutorial/result_voc.pkl tutorial/voc_confusion